In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
%cd ..
from src.data_generators import *
%cd -

/home/ec2-user/SageMaker/data-science-product-image
/home/ec2-user/SageMaker/data-science-product-image/notebooks


Using TensorFlow backend.


In [2]:
sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

input_channels = {
    "train": "s3://data-science-training-datasets/product-image/mturk-30k-plus-10k"
}

In [3]:
hyperparameters={
    "dropout": 0.4,
    "batch_size": 32,
    "num_epochs": 10,
    "num_finetune_epochs": 5,
    "learning_rate": 1e-4,
    "num_lock_layers": 100,
    "validation_split": 0.1,
    "weight_decay": 0.01,
    "max_count": 1000,
    "rotation_range": 5,
    'brightness_range': 0.05,
    'shear_range': 0.2,
    'zoom_range': 0.05,
    'channel_shift_range': 0.05,
    'horizontal_flip': True  
}

In [4]:
estimator = TensorFlow(
    entry_point='concatenated_model_trainer.py',
    source_dir='../src',
    dependencies=['../../data-science-core/util', '../files', '../src'],
    role=role,
    instance_count=1,
    framework_version='2.1.0',
    hyperparameters=hyperparameters, 
    instance_type='ml.p2.xlarge', #'local'
    py_version='py3',
    script_mode=True
)

In [5]:
hyperparameter_ranges = {
    'learning_rate': ContinuousParameter(1e-5, 5e-3),
    'dropout': ContinuousParameter(0.3, 0.5),
    'batch_size': CategoricalParameter([16, 32, 64]),
    'weight_decay': ContinuousParameter(0.001, 0.5),
    'max_count': IntegerParameter(200, 1000),
    'rotation_range': IntegerParameter(4, 15),
    'brightness_range': ContinuousParameter(0.01, 0.2),
    'shear_range': ContinuousParameter(0.01, 0.2),
    'zoom_range': ContinuousParameter(0.01, 0.2),
    'channel_shift_range': ContinuousParameter(0.01, 0.2),
    'num_lock_layers': IntegerParameter(50, 150),
    'combination_type': CategoricalParameter([0, 1])
}

objective_metric_name = 'val_accuracy'
objective_type = 'Maximize'

metric_definitions = [{'Name': objective_metric_name,
                       'Regex': 'Test-Accuracy: ([0-9\\.]+);'}]

In [6]:
tuner = HyperparameterTuner(estimator,
                           objective_metric_name,
                           hyperparameter_ranges,
                           metric_definitions,
                           max_jobs=15,
                           max_parallel_jobs=5,
                           objective_type=objective_type)

In [ ]:
tuner.fit(input_channels)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................